In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
import uuid

In [ ]:
base_cmd = "wsl" # can be empty for local use
exe_dir = "/mnt/c/Users/janhe/datalab/code/" # directory where the executable is found
output_dir = "/mnt/c/Users/janhe/datalab/solutions/exercise1" # directory to save the data file
data_dir = "/mnt/c/Users/janhe/datalab/solutions/exercise1" # directory for jupyter to find the data in

In [ ]:
# Call generate source with given parameters
# Have some default values
def generate_source(
    outputFile=None, 
    sigma=0.25, 
    delta_T=None, 
    N=100000,
    f=0, 
    phi=0, 
    A=0, 
    r=0
):
    """
    Usage: ./generate_source  [-h] [-o <file>] [-s <float>] [-d <float>] [-N <int>] [-f <float>] [-p <float>] [-a <float>] [-r <int>]
      -o, --output=<file>       output file
      -s, --sigma=<float>       standard deviation sigma
      -d, --deltaT=<float>      delta T betwean measurements
      -N, --length=<int>        number of elements
      -f, --freq=<float>        frequency
      -p, --phase=<float>       phase
      -a, --amplitude=<float>   signal amplitude
      -r, --random=<int>        if set to 1, generate random signal
      -h, --help                print this help and exit
    """
    if delta_T == None : 
        delta_T = 1/N
    if outputFile == None:
        outputFile = "data_" + str(uuid.uuid4()) + ".csv"
    
    outputPath = os.path.join(output_dir, outputFile) # output_dir + "/" + "myfilename.csv"
    exe_path = os.path.join(exe_dir, 'generate_source')
    cmd_str = f"{base_cmd} {exe_path} -o {outputPath} -s {sigma} "\
        f"-d {delta_T} -N {N} -f {f} -p {phi} -a {A} -r {r}"
    
    # Optionally print the command line and test it outside the notebook
    #print(cmd_str)
    
    cmd = cmd_str.split(' ')
    process = subprocess.Popen(cmd,stdout=subprocess.PIPE)
    out, err = process.communicate()
    if err:
        print('The process raised an error:', err.decode())
    if out:
        print('The process has an output:', out.decode())

    return os.path.join(data_dir, outputFile)

In [ ]:
def prober(i, m, f=0, p=0, o=None, t=None):
    if o == None:
        o = "results_" + str(uuid.uuid4()) + ".csv"
    
    #path of the executable:
    exe_path = os.path.join(exe_dir, 'prober')
    
    #command line: "{base_cmd} " taken out before "{exe_path}"
    cmd_str = f"{base_cmd} {exe_path} -i {output_dir}/{i} "\
        f"-o {output_dir}/{o} -m {m}"
    
    if t:
        cmd_str += f" -t {output_dir}/{t} "
    else:
        cmd_str += f" -f {f} -p {p}"
    
    # Optionally print the command line and test it outside the notebook
    #print(cmd_str)
    
    cmd = cmd_str.split(' ')
    process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
    out, err = process.communicate()
    if err:
        print('The process raised an error:', err.decode())
    if out:
        print('The process has an output:', out.decode())
    if os.path.isfile(o):
        print(f"Output generated in file {o}")

    return os.path.join(data_dir, o)